Cross validation and bootstrap.

Training error vs test error
Training error is the error we get from applying the model to the data we used to build the model.
Test error is the error we get from applying the model to data that has not been used to build the model.

CP statistic AIC, BIC, use to estimate test error from training error.

Cross-validation, hold out some part of data from training set to use as test data.  Measured by MSE (mean squared error) for quantitative responses and misclassification error rate for qualitative responses (classifications)

CV is used to pick polynomial of the model, not good at telling us the level of the error, wide range.  Highly variable because we're splitting into 2 parts, throwing away half the data in training.

Generally more observations lead to lower error rate.

5.1.1 LOOCV (Leave One Out Cross Validation)
make multiple training sets, each with one value less in it.  Train and test each successive subset and sum the MSE across all the permutations of training/test sets.

Advantage is less bias in the result, and no randomness.  Disadvantage is that it can be expensive to implement


### 1. 
Using basic statistical properties of the variance, as well as single- variable calculus, derive (5.6). In other words, prove that α given by (5.6) does indeed minimize $Var(αX + (1 − α)Y )$.

### 2. 
We will now derive the probability that a given observation is part of a bootstrap sample. Suppose that we obtain a bootstrap sample from a set of n observations.

(a) What is the probability that the first bootstrap observation is not the jth observation from the original sample? Justify your answer.